# Redis Cloud Operations Tutorial

This notebook demonstrates how to connect to Redis Cloud and perform JSON operations with nested data structures.

## Overview
- Connect to Redis Cloud instance
- Generate JSON data with nested properties
- Store data in Redis
- Recursively read and display data
- Verify data integrity

## Step 1: Import Required Libraries

First, we'll import all the necessary libraries for Redis operations and data generation.

In [ ]:
import redis
import json
import random
from datetime import datetime, timedelta

print("Libraries imported successfully!")

## Step 2: Define Redis Connection Function

This function establishes a connection to our Redis Cloud instance using the provided credentials.

In [ ]:
def connect_to_redis():
    """Connect to Redis Cloud instance"""
    try:
        client = redis.Redis(
            host='redis-13092.c282.east-us-mz.azure.redns.redis-cloud.com',
            port=13092,
            username='default',
            password='pzSq7pQas9TFd6gMhcypYC70HRdyyvRv',
            decode_responses=True
        )
        client.ping()
        return client
    except redis.ConnectionError:
        print("Failed to connect to Redis Cloud. Check connection details.")
        return None

print("Redis connection function defined.")

## Step 3: Test Redis Connection

Let's test our connection to make sure everything is working properly.

In [ ]:
# Test the connection
redis_client = connect_to_redis()

if redis_client:
    print("✓ Successfully connected to Redis Cloud!")
    print(f"✓ Redis server info: {redis_client.info()['redis_version']}")
else:
    print("✗ Failed to connect to Redis")

## Step 4: Read Existing JSON Data from Redis

Now let's read an existing JSON object from the Redis database using a specific key.

In [ ]:
def read_redis_data(client, key):
    """
    Read data from Redis, automatically detecting the data type.
    
    Args:
        client: Redis client connection
        key: The Redis key to read
        
    Returns:
        The data stored at the key, or None if not found
    """
    try:
        # Check what type of data structure this key holds
        key_type = client.type(key)
        print(f"Key type: {key_type}")
        
        if key_type == "string":
            # It's a simple string, use GET
            json_data = client.get(key)
            report_data = json.loads(json_data)
            
        elif key_type == "hash":
            # It's a hash, use HGETALL to get all fields
            report_data = client.hgetall(key)
            
        elif key_type == "ReJSON-RL":
            # It's stored using RedisJSON module, use JSON.GET
            report_data = client.execute_command('JSON.GET', key)
            if isinstance(report_data, str):
                report_data = json.loads(report_data)
                
        else:
            print(f"✗ Unsupported key type: {key_type}")
            return None
        
        return report_data
        
    except Exception as e:
        print(f"✗ Error reading data: {e}")
        return None

print("Read function defined.")

In [ ]:
# Call the function to read the report definition
key = "mstr_test:REPORT_DEFINITION:0F1899AD4CAC08A8BA6A979644DF02A4"

report_data = read_redis_data(redis_client, key)

if report_data:
    print(f"✓ Successfully retrieved data for key: {key}")
    print("\n=== Report Definition Data ===")
    print(json.dumps(report_data, indent=2))
else:
    print(f"✗ No data found for key: {key}")

## Step 5: Create RediSearch Index with JSON Path Support

We'll use RediSearch with secondary indexes to enable efficient querying of JSON documents. The index will support full JSON path specification, allowing us to:
- Index top-level 'key' fields that uniquely identify each JSON entry
- Index nested 'key' fields that reference related JSON entries
- Query and retrieve objects efficiently without scanning all keys

In [ ]:
def create_report_index(client, index_name="idx:report_definitions"):
    """
    Create a RediSearch index for REPORT_DEFINITION JSON documents.
    Uses JSON path syntax to index both top-level and nested 'key' fields.
    Only creates the index if it doesn't already exist.
    
    Args:
        client: Redis client connection
        index_name: Name for the index
        
    Returns:
        True if successful or already exists, False otherwise
    """
    try:
        # Check if index already exists
        try:
            info = client.execute_command('FT.INFO', index_name)
            print(f"✓ Index '{index_name}' already exists")
            
            # Count indexed documents
            search_result = client.execute_command('FT.SEARCH', index_name, '*', 'LIMIT', '0', '0')
            doc_count = search_result[0] if search_result else 0
            print(f"Indexed documents: {doc_count}")
            
            return True
            
        except Exception as e:
            # Index doesn't exist, proceed to create it
            print(f"Index does not exist, creating new index: {index_name}")
        
        # Create the index with JSON path specifications
        # ON JSON: Index JSON documents
        # PREFIX: Only index keys starting with this pattern
        # SCHEMA: Define which fields to index and their types
        
        create_command = [
            'FT.CREATE', index_name,
            'ON', 'JSON',
            'PREFIX', '1', 'mstr_test:REPORT_DEFINITION:',
            'SCHEMA',
            # Top-level 'key' field - uniquely identifies this report
            '$.key', 'AS', 'report_key', 'TAG',
            # Example nested paths (adjust based on your actual JSON structure):
            # Nested 'key' fields that reference other objects
            '$..attributes[*].key', 'AS', 'attribute_keys', 'TAG',
            '$..elements[*].key', 'AS', 'element_keys', 'TAG',
            '$..metrics[*].key', 'AS', 'metric_keys', 'TAG',
            # Add other commonly queried fields
            '$.name', 'AS', 'name', 'TEXT',
            '$.id', 'AS', 'id', 'TAG'
        ]
        
        result = client.execute_command(*create_command)
        print(f"✓ Successfully created index: {index_name}")
        print(f"Index will automatically index all existing and new documents with prefix: mstr_test:REPORT_DEFINITION:")
        
        # Count indexed documents
        search_result = client.execute_command('FT.SEARCH', index_name, '*', 'LIMIT', '0', '0')
        doc_count = search_result[0] if search_result else 0
        print(f"Indexed documents: {doc_count}")
        
        return True
        
    except Exception as e:
        print(f"✗ Error creating index: {e}")
        return False

# Create the index (only if it doesn't exist)
create_report_index(redis_client)

## Step 6: Extract All 'key' Values from JSON Document

Now let's extract and categorize all 'key' values found in our report definition. We'll differentiate between:
- Top-level 'key' that identifies the document itself
- Nested 'key' values that reference related objects

In [ ]:
def extract_all_keys(data, parent_path="", keys_dict=None):
    """
    Recursively extract all 'key' fields from a JSON document.
    Tracks the path to each key to differentiate between top-level and nested keys.
    
    Args:
        data: JSON data (dict or list)
        parent_path: Current path in the JSON structure
        keys_dict: Dictionary to store found keys with their paths
        
    Returns:
        Dictionary with paths as keys and 'key' values as values
    """
    if keys_dict is None:
        keys_dict = {
            'top_level': None,
            'nested': []
        }
    
    if isinstance(data, dict):
        for key, value in data.items():
            current_path = f"{parent_path}.{key}" if parent_path else key
            
            # Check if this is a 'key' field
            if key == 'key':
                if parent_path == "":
                    # Top-level key
                    keys_dict['top_level'] = value
                else:
                    # Nested key
                    keys_dict['nested'].append({
                        'path': parent_path,
                        'value': value
                    })
            
            # Recurse into nested structures
            if isinstance(value, (dict, list)):
                extract_all_keys(value, current_path, keys_dict)
                
    elif isinstance(data, list):
        for i, item in enumerate(data):
            current_path = f"{parent_path}[{i}]"
            if isinstance(item, (dict, list)):
                extract_all_keys(item, current_path, keys_dict)
    
    return keys_dict

# Extract all keys from the report data we retrieved earlier
if report_data:
    all_keys = extract_all_keys(report_data)
    
    print("=== Key Values Found ===\n")
    
    # Display top-level key
    print(f"Top-level key (identifies this document):")
    if all_keys['top_level']:
        print(f"  {all_keys['top_level']}")
    else:
        print(f"  (None found)")
    
    # Display nested keys
    print(f"\nNested keys (references to related objects): {len(all_keys['nested'])} found")
    if all_keys['nested']:
        # Group by path for better organization
        paths = {}
        for item in all_keys['nested']:
            path = item['path']
            if path not in paths:
                paths[path] = []
            paths[path].append(item['value'])
        
        for path, values in paths.items():
            print(f"\n  Path: {path}")
            for value in values:
                print(f"    - {value}")
    
    # Create a flat list of all nested key values (useful for querying)
    nested_key_values = [item['value'] for item in all_keys['nested']]
    print(f"\n=== Summary ===")
    print(f"Total nested key values: {len(nested_key_values)}")
    print(f"Unique nested key values: {len(set(nested_key_values))}")
    
else:
    print("No report data available. Please run Step 4 first.")

## Step 7: Recursively Read Related JSON Documents

Now we'll use the nested 'key' values to read all related JSON documents from Redis. For each document found, we'll extract all its 'key' values as well, building a complete map of relationships.

In [ ]:
def read_related_documents(client, nested_keys, prefix="mstr_test"):
    """
    Read all JSON documents referenced by nested key values.
    Uses the key values directly as Redis keys without inferring document types.
    Does not recursively read nested documents.
    
    Args:
        client: Redis client connection
        nested_keys: List of nested key dictionaries from extract_all_keys()
        prefix: Redis key prefix to use when constructing full keys
        
    Returns:
        Dictionary mapping key values to their extracted data and nested keys
    """
    related_docs = {}
    
    # Get unique key values to avoid duplicate reads
    unique_keys = list(set([item['value'] for item in nested_keys]))
    
    print(f"Reading {len(unique_keys)} related documents...")
    print("=" * 60)
    
    for key_value in unique_keys:
        # Construct the full Redis key using the prefix and key value
        redis_key = f"{prefix}:{key_value}"
        
        # Try to read the document
        doc_data = read_redis_data(client, redis_key)
        
        if doc_data:
            # Extract all keys from this document (but don't read them recursively)
            doc_keys = extract_all_keys(doc_data)
            
            related_docs[key_value] = {
                'redis_key': redis_key,
                'data': doc_data,
                'top_level_key': doc_keys['top_level'],
                'nested_keys': doc_keys['nested'],
                'nested_key_count': len(doc_keys['nested'])
            }
            
            print(f"\n✓ Found: {key_value}")
            print(f"  Redis key: {redis_key}")
            print(f"  Has top-level key: {doc_keys['top_level'] is not None}")
            print(f"  Nested keys found: {len(doc_keys['nested'])}")
            
            # List out the nested key values (but don't read them)
            if doc_keys['nested']:
                nested_values = [item['value'] for item in doc_keys['nested']]
                unique_nested = list(set(nested_values))
                print(f"  Nested key values: {', '.join(unique_nested[:5])}")
                if len(unique_nested) > 5:
                    print(f"    ... and {len(unique_nested) - 5} more")
        else:
            print(f"\n✗ Not found: {key_value}")
            print(f"  Tried Redis key: {redis_key}")
            related_docs[key_value] = None
    
    return related_docs

# Read all related documents from the nested keys we found
if report_data and all_keys['nested']:
    print("=== Reading Related Documents ===\n")
    related_documents = read_related_documents(redis_client, all_keys['nested'])
    
    # Summary
    print("\n" + "=" * 60)
    print("=== Summary ===")
    found_count = sum(1 for doc in related_documents.values() if doc is not None)
    not_found_count = sum(1 for doc in related_documents.values() if doc is None)
    
    print(f"Documents found: {found_count}")
    print(f"Documents not found: {not_found_count}")
    print(f"Total unique key values searched: {len(related_documents)}")
    
    # Count total nested keys in all related documents
    total_nested_keys = sum(doc['nested_key_count'] for doc in related_documents.values() if doc is not None)
    print(f"Total nested keys across all related documents: {total_nested_keys}")
    
    # List all documents that were successfully read
    if found_count > 0:
        print(f"\n=== Successfully Retrieved Documents ===")
        for key_val, doc_info in related_documents.items():
            if doc_info is not None:
                print(f"  - {key_val} (Redis key: {doc_info['redis_key']})")
    
else:
    print("No nested keys found in the original report data.")
    print("Please ensure Step 6 has been run successfully.")